In [26]:
import gc
import torch
import pandas as pd

from models.load import TrainedModels, get_trained_model
from data.constants import XAMI_MIMIC_PATH, DEFAULT_REFLACX_LABEL_COLS
from utils.coco_utils import get_coco_api_from_dataset

from utils.init import reproducibility, clean_memory_get_device
from data.datasets import ReflacxDataset, collate_fn
from data.transforms import get_transform
from utils.coco_eval import get_eval_params_dict

import pandas as pd
import numpy as np

import utils.print as print_f

from utils.coco_eval import get_eval_params_dict
from utils.engine import xami_evaluate, get_iou_types

from utils.eval import get_ap_ar
from utils.init import reproducibility, clean_memory_get_device
from data.constants import DEFAULT_REFLACX_LABEL_COLS, XAMI_MIMIC_PATH

## Suppress the assignement warning from pandas.
pd.options.mode.chained_assignment = None  # default='warn

## Supress user warning
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

%matplotlib inline

In [31]:
from enum import Enum

class TrainedModels(Enum):
    CXR_200_ex="val_ar_0_0000_ap_0_0000_test_ar_0_0000_ap_0_0000_epoch5_08-29-2022 21-46-47_forward_testing_model"
    CXR_only_images= "val_ar_-1_0000_ap_-1_0000_test_ar_0_0000_ap_0_0000_epoch10_08-29-2022 17-54-50_forward_testing_model"
    CXR_6_hours= "val_ar_0_0016_ap_0_0017_test_ar_0_0000_ap_0_0000_epoch2_08-23-2022 23-36-16_forward_testing_model"

In [32]:
use_iobb = True
io_type_str = "IoBB" if use_iobb else "IoU"
labels_cols = DEFAULT_REFLACX_LABEL_COLS
iou_thrs = np.array([0.5])


device = clean_memory_get_device()
reproducibility()

This notebook will running on device: [CPU]


In [33]:
model, train_info, _, _ = get_trained_model(
    TrainedModels.CXR_200_ex,
    DEFAULT_REFLACX_LABEL_COLS,
    device,
    rpn_nms_thresh=0.3,
    box_detections_per_img=10,
    box_nms_thresh=0.2,
    rpn_score_thresh=0.0,
    box_score_thresh=0.05,
)

model.to(device)



Using pretrained backbone. mobilenet_v3
Using pretrained backbone. mobilenet_v3
forward_testing_model will use mask, [64] layers.
Using SGD as optimizer with lr=0.001


MultimodalMaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): Sequential(
    (0): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): Hardswish()
      )
      (1): InvertedResidual(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
            (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))


In [34]:
print(train_info)

========================================For Training [forward_testing_model]========================================
ModelSetup(name='forward_testing_model', use_fixations=True, save_early_stop_model=True, record_training_performance=True, backbone='mobilenet_v3', optimiser='sgd', lr=0.001, weight_decay=1e-05, image_backbone_pretrained=True, fixation_backbone_pretrained=True, image_size=512, backbone_out_channels=64, batch_size=4, warmup_epochs=0, lr_scheduler='ReduceLROnPlateau', reduceLROnPlateau_factor=0.1, reduceLROnPlateau_patience=999, reduceLROnPlateau_full_stop=True, multiStepLR_milestones=100, multiStepLR_gamma=0.1, representation_size=64, mask_hidden_layers=64, using_fpn=False, use_mask=True, fuse_conv_channels=64, box_head_dropout_rate=0, fuse_depth=0, fusion_strategy='add', fusion_residule=False, gt_in_train_till=999, measure_test=True)

Best AP validation model has been saved to: [val_ar_0_0833_ap_0_0040_test_ar_0_0526_ap_0_0010_epoch2_08-29-2022 21-14-48_forward_testing_m

In [35]:
model

MultimodalMaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): Sequential(
    (0): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): Hardswish()
      )
      (1): InvertedResidual(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
            (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))


In [36]:
train_info.best_ap_val_model_path

'val_ar_0_0833_ap_0_0040_test_ar_0_0526_ap_0_0010_epoch2_08-29-2022 21-14-48_forward_testing_model'

In [37]:
use_iobb = True
io_type_str = "IoBB" if use_iobb else "IoU"
labels_cols = DEFAULT_REFLACX_LABEL_COLS
iou_thrs = np.array([0.5])

In [38]:
model_setup = train_info.model_setup

dataset_params_dict = {
    "XAMI_MIMIC_PATH": XAMI_MIMIC_PATH,
    "bbox_to_mask": model_setup.use_mask,
    "labels_cols": DEFAULT_REFLACX_LABEL_COLS,
    "with_fixation": True
}


test_dataset = ReflacxDataset(
    **dataset_params_dict, split_str="test", transforms=get_transform(train=False),
)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn,
)

test_coco = get_coco_api_from_dataset(test_dataloader.dataset)

eval_params_dict = get_eval_params_dict(
        test_dataset, iou_thrs=iou_thrs, use_iobb=use_iobb,
    )

iou_types = get_iou_types(model, train_info.model_setup)

# all_range_eval_params_dict = get_eval_params_dict(
#     test_dataset, iou_thrs=full_iou_thrs,
# )


creating index...
index created!
creating index...
index created!


In [39]:
# model.roi_heads.use_gt_in_train = False
model.roi_heads.score_thresh = 0.3
test_evaluator, test_logger = xami_evaluate(
            model=model,
            data_loader=test_dataloader,
            device=device,
            params_dict=eval_params_dict,
            coco=test_coco,
            iou_types=iou_types,
            score_thres=None,
        )

Evaluation:  [ 0/18]  eta: 0:00:38  loss: 0.4942 (0.4942)  loss_classifier: 0.0309 (0.0309)  loss_box_reg: 0.0112 (0.0112)  loss_mask: 0.3999 (0.3999)  loss_objectness: 0.0478 (0.0478)  loss_rpn_box_reg: 0.0044 (0.0044)  model_time: 0.7969 (0.7969)  evaluator_time: 0.0030 (0.0030)  time: 2.1253  data: 1.3255
Evaluation:  [17/18]  eta: 0:00:02  loss: 0.0509 (0.2673)  loss_classifier: 0.0103 (0.0197)  loss_box_reg: 0.0000 (0.0082)  loss_mask: 0.0000 (0.1978)  loss_objectness: 0.0323 (0.0387)  loss_rpn_box_reg: 0.0022 (0.0029)  model_time: 1.1300 (1.0578)  evaluator_time: 0.0020 (0.0024)  time: 2.6545  data: 1.5572
Evaluation: Total time: 0:00:47 (2.6547 s / it)
Averaged stats: loss: 0.0509 (0.2673)  loss_classifier: 0.0103 (0.0197)  loss_box_reg: 0.0000 (0.0082)  loss_mask: 0.0000 (0.1978)  loss_objectness: 0.0323 (0.0387)  loss_rpn_box_reg: 0.0022 (0.0029)  model_time: 1.1300 (1.0578)  evaluator_time: 0.0020 (0.0024)
Accumulating evaluation results...
DONE (t=0.01s).
Accumulating evalua

In [40]:
test_ap_ar = get_ap_ar(
    test_evaluator, areaRng="all", iouThr=0.5, maxDets=10,
)

test_ap_ar

{'ap': 0.0, 'ar': 0.0}